In [ ]:
#default_exp main

# Metadados e Espectro
> Módulo com funções que encapsulam o processamento completo e retorno de metadados, matrix de nível.

In [ ]:
#hide 
#Standard Lib imports
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [ ]:
#hide
%load_ext autoreload
%autoreload 2            #Reload the code automatically
%config Completer.use_jedi = False

In [ ]:
#export
from datetime import datetime
from typing import Union, Iterable
import os

from fastcore.basics import typed
from fastcore.xtras import Path
from fastcore.xtras import is_listy
from fastcore.foundation import L
import numpy as np
import pandas as pd

from rich import print
from loguru import logger
from rfpye.utils import *
from rfpye.constants import SPECTRAL_BLOCKS, console
from rfpye.parser import parse_bin, export_metadata

BTYPES = [21, 40] + SPECTRAL_BLOCKS
FILTER_ATTRS = {21: ['hostname', 'method'],
                40: ['altitude', 'latitude', 'longitude'],
                63: ['wallclock_datetime', 'sample', 'minimum', 'ndata', 'processing', 'bw', 'unit', 'description', 'start_mega', 'stop_mega'],
                67: ['wallclock_datetime', 'sample', 'minimum', 'ndata', 'processing', 'bw', 'unit', 'description', 'start_mega', 'stop_mega'],
                68: ['wallclock_datetime', 'sample', 'minimum', 'ndata', 'processing', 'bw', 'unit', 'description', 'start_mega', 'stop_mega']}

In [ ]:
#exporti
logger.add("rfpye.log", rotation="1 week", compression='zip', backtrace=True, diagnose=True) 

2

In [ ]:
#export
def parse_path(path: Union[list, str])->Iterable:
    if is_listy(path):
        return [f for f in L(path).map(Path) if f.is_file() and f.suffix == '.bin']
    if isinstance(path, (str, Path)):
        path = Path(path)
        if path.is_file() and path.suffix == '.bin':
                return [path]
        elif path.is_dir():
            return get_files(path, extensions=[".bin"])        
    raise ValueError(f"Caminho de Entrada inválido: {path}. Insira um caminho para uma pasta, um arquivo ou lista de arquivos")

In [ ]:
#export
@logger.catch
def extract_bin_data(
    path: Union[list, str],
    meta_attrs: Iterable = None,
    spec_data: bool = False,
    spec_dtype: str = "float16",
    
) -> None:
    """Recebe uma pasta ou arquivo bin, processa e salva os metadados e espectro na saida.

    Args:
        path (str): Arquivo ou Lista de Arquivos .bin | Caminho para a Pasta
        spec_data (bool, optional): Retornar dados de Espectro?. Defaults to False.
        dtype (str, optional): Tipo de dados a salvar o espectro. Defaults to "float16".
    """
    lista_bins = parse_path(path)

    console.rule("Lista de Arquivos a serem processados", style="bold red")
    console.print(
        [f.name for f in lista_bins],
        style="bold white",
        overflow="fold",
        justify="left",
    )
    if not lista_bins:
        console.print(":sleeping: Nenhum arquivo .bin a processar :zzz:")
        return
    
    output = []
    for file in lista_bins:
        console.print(f"[green]Processando Blocos de: [red]{file.name}")
        parsed_bin = parse_bin(file, btypes=BTYPES)
        file_version, blocks = parsed_bin.values()
        metadata = export_metadata(blocks, filter_attrs=FILTER_ATTRS)
        out = dict()
        out['File_Name'] = file.name
        out['File_Version'] = file_version
        out['File_Type'] = 'RFEye Logger Trace'
        out['Device'] = 'Rfeye Node'
        out['Fluxos'] = dict()
        for (btype, tid), df in metadata.items():
            if btype == 21:
                out['Equipment_ID'] = df.hostname.item()
                out['Script_Version'] = df.method.item()
            elif btype == 40:
                out['Latitude'] = df.latitude.median()
                out['Longitude'] = df.longitude.median()
                out['Altitude'] = df.altitude.median()
                out['Count_GPS'] = df.shape[0]
                out['Sum_Latitude']  = df.latitude.sum()
                out['Sum_Longitude'] = df.longitude.sum()
            elif btype in SPECTRAL_BLOCKS:
                timestamp = df.index.values
                level = dict()
                level['Initial_Time'] = timestamp.min()
                level['Sample_Duration'] = df['sample'].median()
                fluxo = df.drop(['minimum', 'sample'], axis=1).iloc[0]
                level['Description'] = fluxo.description
                level['Start_Frequency'] = fluxo.start_mega
                level['Stop_Frequency'] = fluxo.stop_mega
                level['Trace_Type'] = fluxo.processing
                level['Resolution'] = fluxo.bw
                level['Level_Units'] = fluxo.unit
                level['Num_Traces'] = df.shape[0]
                level['Vector_Length'] = fluxo.ndata
                level['Timestamp'] = timestamp
                if spec_data:
                    if save_path:
                        level['Minimum_Level'] = df.minimum.values.astype('float16')
                        level['Level_Data'] = extract_level(blocks[(btype, tid)], dtype=np.uint8).flatten()
                    else:
                        level['Frequency'] = np.linspace(fluxo.start_mega, fluxo.stop_mega, num=fluxo.ndata)
                        level['Level_Data'] = extract_level(blocks[(btype, tid)], dtype=dtype)
                out['Fluxos'][(btype, tid)] =  level
            else:
                print(btype)
        done.add(file.name)
        output[file.name] = out

    if save_path:
        log.write_text("\n".join(sorted(list(done))))
    return output

Uso mais comum: Diretório com arquivos CRFS bin
A função seguinte recebe um caminho para um arquivo `.bin` ou para uma pasta que contém arquivos bin e retorna os metadados do arquivo como um dicionário. 

In [ ]:
path = Path(r'D:\OneDrive - ANATEL\BinFiles\Combo3 (Tipos de dados 4, 7, 8, 60-65 e 67-69)')

In [ ]:
if not path.exists() or not len(get_files(path, extensions=['.bin'])):
    path = Path('.')
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T202310_CRFSBINv.5.bin' --output-document 'rfeye002092_210208_T202310_CRFSBINv.5.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002092_210208_T203131_CRFSBINv.2.bin' --output-document 'rfeye002092_210208_T203131_CRFSBINv.2.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T202215_CRFSBINv.4.bin' --output-document 'rfeye002292_210208_T202215_CRFSBINv.4.bin'
    !wget --header 'Host: raw.githubusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: pt-BR,pt;q=0.8,en-US;q=0.5,en;q=0.3' --referer 'https://github.com/EricMagalhaesDelgado/SpecFiles/blob/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --header 'DNT: 1' --header 'Upgrade-Insecure-Requests: 1' 'https://raw.githubusercontent.com/EricMagalhaesDelgado/SpecFiles/main/Combo3%20(CRFS%20Bin%20-%20DataTypes%204%2C%207%2C%208%2C%2060-65%20e%2067-69)/rfeye002292_210208_T203238_CRFSBINv.3.bin' --output-document 'rfeye002292_210208_T203238_CRFSBINv.3.bin'


In [ ]:
dados = extract_bin_data(path)

─────────────────────────── Lista de Arquivos a serem processados ───────────────────────────

[                                                                                            
    'rfeye002092_210208_T202310_CRFSBINv.5.bin',                                             
    'rfeye002092_210208_T203131_CRFSBINv.2.bin',                                             
    'rfeye002292_210208_T202215_CRFSBINv.4.bin',                                             
    'rfeye002292_210208_T203238_CRFSBINv.3.bin'                                              
]                                                                                            

Processando Blocos de: rfeye002092_210208_T202310_CRFSBINv.5.bin

2021-10-07 16:13:14.648 | ERROR    | __main__:<module>:1 - An error has been caught in function '<module>', process 'MainProcess' (11816), thread 'MainThread' (22380):
Traceback (most recent call last):

  File "C:\Users\rsilva\Miniconda3\envs\rfpye\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
                └ ModuleSpec(name='ipykernel_launcher', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000291A84EEF48>, orig...

  File "C:\Users\rsilva\Miniconda3\envs\rfpye\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code object <module> at 0x00000291A8DB3270, file "C:\Users\rsilva\Miniconda3\envs\rfpye\lib\site-packages\ipykernel_launcher...

  File "C:\Users\rsilva\Miniconda3\envs\rfpye\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_in

In [ ]:
for filename, dado in dados.items():
    print(f'Metadados do Arquivo {filename}')
    print({k:v for k,v in dado.items() if k != 'Fluxos'})
    print("Fluxos: ", {k:{i:j for i,j in fluxo.items()  if i not in ('Timestamp', 'Frequency', 'Level_Data')} for k,fluxo in dado['Fluxos'].items()})

Metadados do Arquivo rfeye002092_210208_T202310_CRFSBINv.5.bin

{
    'File_Name': 'rfeye002092_210208_T202310_CRFSBINv.5.bin',
    'File_Version': 23,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node',
    'Equipment_ID': 'rfeye002092',
    'Script_Version': 'Script_CRFSBINv5',
    'Latitude': -12.826857,
    'Longitude': -38.478035,
    'Altitude': 124.3,
    'Count_GPS': 8,
    'Sum_Latitude': -102.61485,
    'Sum_Longitude': -307.82428
}

Fluxos: 
{
    (67, 20): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:10.500680000'),
        'Sample_Duration': 18073.0,
        'Description': 'ClearWrite. Peak.',
        'Start_Frequency': 76,
        'Stop_Frequency': 108,
        'Trace_Type': 'peak',
        'RBW': 7381,
        'Level_Units': 'dBm',
        'Num_Traces': 956,
        'Vector_Length': 8192
    },
    (67, 30): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:10.522721000'),
        'Sample_Duration': 8552.0,
        'Description': 'ClearWrite. Peak.',
        'Start_Frequency': 70,
        'Stop_Frequency': 110,
        'Trace_Type': 'peak',
        'RBW': 73828,
        'Level_Units': 'dBm',
        'Num_Traces': 988,
        'Vector_Length': 1024
    },
    (67, 10): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:11.979000000'),
        'Sample_Duration': 8076.0,
        'Description': 'ClearWrite. Peak.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 477,
        'Vector_Length': 3584
    },
    (67, 11): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:11.979000000'),
        'Sample_Duration': 8076.0,
        'Description': 'ClearWrite. Mean.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 477,
        'Vector_Length': 3584
    },
    (67, 12): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:20.801000000'),
        'Sample_Duration': 8081.0,
        'Description': 'MaxHold. Peak. Count = 10.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 26,
        'Vector_Length': 3584
    },
    (68, 14): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:20.801000000'),
        'Sample_Duration': 8081.0,
        'Description': 'Average. Mean. Count = 10.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 49,
        'Vector_Length': 3584
    },
    (67, 13): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:23:30.763000000'),
        'Sample_Duration': 8080.5,
        'Description': 'Average. Mean. Count = 10.',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 18457,
        'Level_Units': 'dBm',
        'Num_Traces': 32,
        'Vector_Length': 3584
    }
}

Metadados do Arquivo rfeye002092_210208_T203131_CRFSBINv.2.bin

{
    'File_Name': 'rfeye002092_210208_T203131_CRFSBINv.2.bin',
    'File_Version': 21,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node'
}

...

{
    'File_Name': 'rfeye002292_210208_T202215_CRFSBINv.4.bin',
    'File_Version': 22,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node',
    'Equipment_ID': 'rfeye002292',
    'Script_Version': 'Script_CRFSBINv4',
    'Latitude': -12.971631,
    'Longitude': -38.48149,
    'Altitude': 151.65,
    'Count_GPS': 8,
    'Sum_Latitude': -103.773056,
    'Sum_Longitude': -307.85193
}

Fluxos: 
{
    (63, 20): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:15.500658000'),
        'Sample_Duration': 18140.0,
        'Description': 'Peak',
        'Start_Frequency': 76,
        'Stop_Frequency': 108,
        'Trace_Type': 'peak',
        'RBW': 3,
        'Level_Units': 'dBm',
        'Num_Traces': 1047,
        'Vector_Length': 8192
    },
    (63, 30): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:15.523638000'),
        'Sample_Duration': 8549.0,
        'Description': 'Peak',
        'Start_Frequency': 70,
        'Stop_Frequency': 110,
        'Trace_Type': 'peak',
        'RBW': 39,
        'Level_Units': 'dBm',
        'Num_Traces': 1048,
        'Vector_Length': 1024
    },
    (63, 10): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:16.457000000'),
        'Sample_Duration': 8083.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 524,
        'Vector_Length': 3584
    },
    (63, 11): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:16.457000000'),
        'Sample_Duration': 8083.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 524,
        'Vector_Length': 3584
    },
    (63, 12): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:25.618000000'),
        'Sample_Duration': 8076.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'peak',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 52,
        'Vector_Length': 3584
    },
    (63, 13): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:25.618000000'),
        'Sample_Duration': 8076.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 9,
        'Level_Units': 'dBm',
        'Num_Traces': 52,
        'Vector_Length': 3584
    },
    (64, 14): {
        'Initial_Time': numpy.datetime64('2021-02-08T20:22:25.618000000'),
        'Sample_Duration': 8076.0,
        'Description': 'Peak',
        'Start_Frequency': 105,
        'Stop_Frequency': 140,
        'Trace_Type': 'average',
        'RBW': 48,
        'Level_Units': 'dBm',
        'Num_Traces': 52,
        'Vector_Length': 717
    }
}

Metadados do Arquivo rfeye002292_210208_T203238_CRFSBINv.3.bin

{
    'File_Name': 'rfeye002292_210208_T203238_CRFSBINv.3.bin',
    'File_Version': 21,
    'File_Type': 'RFEye Logger Trace',
    'Device': 'Rfeye Node',
    'Equipment_ID': 'rfeye002292',
    'Script_Version': 'Script_CRFSBINv3',
    'Latitude': -12.971631,
    'Longitude': -38.481487,
    'Altitude': 150.6,
    'Count_GPS': 12,
    'Sum_Latitude': -155.65956,
    'Sum_Longitude': -461.7778
}

Fluxos: 
{}

In [ ]:
dados = extract_bin_data(path / 'rfeye002092_210208_T202310_CRFSBINv.5.bin', spec_data=True)

─────────────────────────── Lista de Arquivos a serem processados ───────────────────────────

['rfeye002092_210208_T202310_CRFSBINv.5.bin']                                                

Processando Blocos de: rfeye002092_210208_T202310_CRFSBINv.5.bin

In [ ]:
d = dados['rfeye002092_210208_T202310_CRFSBINv.5.bin']
fluxo = d['Fluxos'][67,20]
tempo = fluxo['Timestamp']
frequencias = np.linspace(fluxo['Start_Frequency'], fluxo['Stop_Frequency'], num=fluxo['Vector_Length'])
niveis = fluxo['Level_Data'] 
data = pd.DataFrame(niveis, index=tempo, columns=frequencias)
data.columns.name = "Frequencies"
data.index.name = "Time"
print(data.head())

[[-117.5 -113.  -111.5 ... -103.5 -110.   -97.5]
 [-113.5 -114.5 -111.  ...  -94.5  -98.5  -99.5]
 [-109.5 -111.5 -114.  ... -100.   -99.  -100.5]
 ...
 [-117.  -115.  -114.  ...  -99.5  -96.   -99. ]
 [-117.  -118.  -117.5 ...  -99.5  -93.5  -93.5]
 [-114.5 -110.5 -110.  ...  -96.5  -96.  -102. ]]

## Extrair dados estatísticos

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
Converted index.ipynb.
